# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable  #prints output in tabular format

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #pint(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project. 

## located within the Workspace directory.  The <font color=red>event_datafile_new.csv</font>  created above contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Denormalized data sample result <font color=red>**event_datafile_new.csv**</font> from the code above:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Creating a Keyspace 

try: 
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_mv 
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
 )
    
except Expection as e:
    print(e)                  
                    

#### Set Keyspace

In [7]:
#Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_mv')

except exception as e:
    print(e)

## Queries & Implementation

### Addressing queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## <font color=green>Implementation</font>

<h3>Query 1: Give me the artist, song title and songs length in the music app history that was heard during sessionId = 338 and itemInSession = 4</h3>    

### <font color=green>Solution #1</font>

#### 1.1 Create table

#### <font color = green>Primary key approach:</font> 
 ##### <font color = green>Primary key -> combination of SessionId (Partition key) & iteminSession (Clustering column)</font>  
 ##### <font color = green>SessionId: unique value across a session making it an ideal candidate for a partition key </font>
 ##### <font color = green>IteminSession: As clustering column and in combination with SessionID will make each row</font> 
 #####  <font color = green> unique at a session grain, thus making it the best primary key in this scenario. </font>  

 
   

In [8]:
## Query to retrieve song details (artist, title & length) based on Sessionid and iteminsession.

query1 = ("""CREATE TABLE IF NOT EXISTS song_plays_bysession(
 sessionID INT
,iteminSession INT
,artist VARCHAR
,song VARCHAR
,length FLOAT
,PRIMARY KEY((sessionid), iteminSession)
)
""")

try: 
    session.execute(query1)
    
except exception as e: 
    print(e)

#### 1.2 Insert data

In [9]:
## Insert data into the above table, from the event data file created earlier - 'event_datafile_new.csv'

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_plays_bysession(sessionID, iteminSession, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        ## Assigning column element assigned (from the csv file) corresponding to each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### 1.3 Verify with Select statement

In [10]:
## Adding in the SELECT statement to verify the data was entered into the table and 
 # addressing Q1: Qive me the artist,song title and song's length in the music app history 
 # that was heard during sessionId = 338, and itemInSession = 4

query1 = """SELECT artist, song, length  
           FROM song_plays_bysession
           WHERE sessionID = 338
            AND iteminSession = 4
        """
try:
    rows = session.execute(query1)
    
except Exception as e:
    print(e)
    
#Print output in a tabular format using PrettyTable API
t= PrettyTable()
    

#print result
for row in rows:
     t.field_names = ['artist','song', 'length']  #define column names
     t.add_row([row.artist, row.song, row.length])  #iterating through the data in rows
print(t)

+-----------+---------------------------------+--------------------+
|   artist  |               song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


<h3>Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)for userid = 10, sessionid = 182<h3>

### <font color=green>Solution #2</font>

#### 2.1 Create table

#### <font color = green> Primary key approach: </font>
 ##### <font color = green> Primary key -> combination of userid,  sessionid (composite partition key) & iteminSession (Clustering column) </font>
 ##### <font color = green> userid + sessionId:  Have a one to one relationship. Each user is assigned a unique value per session</font>
 ##### <font color = green> IteminSession: Assigning it as a clustering column and then combining it with userid + SessionID</font>
 ##### <font color = green>will make each row unique. The clustering column also comes sorted in ascending order( iteminsession) </font> 

 

In [11]:
## Query to retrieve artist and song(s) played by user in a particular session

query2 = ("""CREATE TABLE IF NOT EXISTS songsplays_by_userandsession(
  userid INT 
 ,sessionid INT
 ,iteminSession INT
 ,firstname VARCHAR
 ,lastname VARCHAR
 ,artist VARCHAR
 ,song VARCHAR
 ,PRIMARY KEY((userid, sessionid), iteminsession)
)
""")

try: 
    session.execute(query2)
    
except exception as e: 
    print(e)
                    

#### 2.2 Insert data

In [12]:
## Insert data into the above table, from the event data file created earlier - 'event_datafile_new.csv'

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO songsplays_by_userandsession(userid, sessionid, iteminSession, firstname, lastname,\
                 artist, song)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assigning column element assigned (from the csv file) corresponding to each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

#### 2.3 Verify with Select statement

In [13]:
## Adding in the SELECT statement to verify the data was entered into the table and
 # addressing Q2: Give me only the following: name of artist, song (sorted by itemInSession) and
 # user (first and last name) for userid = 10, sessionid = 182

query2 = """SELECT artist, song, iteminsession, firstname, lastname
           FROM songsplays_by_userandsession
           WHERE userid = 10
            AND sessionid = 182
         """

try:
    rows = session.execute(query2)
    
except Exception as e:
    print(e)

#Print output in a tabular format using PrettyTable API
t= PrettyTable()


#print result    
for row in rows:
      t.field_names= ['artist','song', 'iteminsession', 'firstname', 'lastname'] #define column names
      t.add_row([row.artist, row.song, row.iteminsession, row.firstname, row.lastname]) #iterating through the data in rows
print(t)

+-------------------+------------------------------------------------------+---------------+-----------+----------+
|       artist      |                         song                         | iteminsession | firstname | lastname |
+-------------------+------------------------------------------------------+---------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |       0       |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |       1       |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |       2       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |       3       |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+---------------+-----------+----------+


<h3>Query 3: Give me every user name (first and last) in my music app history who listened to the song 
'All Hands Against His Own'<h3>

### <font color=green>Solution #3</font>

#### 3.1 Create table

#### <font color = green> Primary key approach: </font>
 ##### <font color = green> Primary key -> combination of song (partition key) & userid (Clustering column) </font>
 ##### <font color = green> Song: Since we want to extract the data by users who listened to a particular song, 'song'  would be the best candidate for a partition key</font>
 ##### <font color = green> userid: Combining it with partition key above, will make it unique to each row at a song-userid grain</font>
 

In [14]:
## Query based on Users who listened to a given song
query3 = ("""CREATE TABLE IF NOT EXISTS users_bysongs(
  userid INT 
 ,firstname VARCHAR
 ,lastname VARCHAR
 ,artist VARCHAR
 ,song VARCHAR
 ,PRIMARY KEY((song), userid)
)
""")

try: 
    session.execute(query3)
    
except exception as e: 
    print(e)

                    

#### 3.2 Insert data

In [15]:
## Insert data into the above table, from the event data file created earlier - 'event_datafile_new.csv'
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
##Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO users_bysongs(userid, firstname, lastname, artist, song)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        ## Assigning column element assigned (from the csv file) corresponding to each column in the INSERT statement.
        session.execute(query, (int(line[10]), line[1], line[4], line[0], line[9]))


#### 3.3 Verify with Select statement

In [16]:
## Adding in the SELECT statement to verify the data was entered into the table and address Q3: Give me every user name 
 # (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = """SELECT firstname, lastname
           FROM users_bysongs
           WHERE song = %s
         """

parameter1 = ('All Hands Against His Own',)


try:
    rows = session.execute(query3, parameter1)
    
except Exception as e:
    print(e)

#Print output in a tabular format using PrettyTable API
t= PrettyTable()

#print result 
for row in rows:
    t.field_names = ['firstname', 'lastname'] #define column names
    t.add_row([row.firstname, row.lastname]) #iterating through the data in rows
    
print(t)


+------------+----------+
| firstname  | lastname |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [17]:
#Table1
query = "DROP TABLE IF EXISTS song_plays_bysession"
try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)      
   

In [18]:
#Table2
query = "DROP TABLE IF EXISTS songsplays_by_userandsession"
try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)    

In [19]:
#Table3
query = "DROP TABLE IF EXISTS users_bysongs"
try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)  

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()